In [3]:
import logging
logging.basicConfig(level=logging.INFO, format='%(message)s')

from model import DataProcessor, FeatureEngineer, ContentRecommender, Evaluator, EmbeddingFeatureEngineer
import os
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

In [4]:
movies_path = 'netflix_movies_detailed_up_to_2025.csv'
tv_path = 'netflix_tv_shows_detailed_up_to_2025.csv'

df = DataProcessor(movies_path, tv_path).process()

fe = FeatureEngineer(df)
fe.create_tag().build_tfidf_matrix().build_separate_matrices()
df, tfidf_matrix, tfidf = fe.get_outputs()

efe_tag = EmbeddingFeatureEngineer(df)
efe_tag.build_embeddings(column='tag')
_, tag_embeddings = efe_tag.get_outputs()

efe_des = EmbeddingFeatureEngineer(df)
efe_des.build_embeddings(column='description')
_, des_embeddings = efe_des.get_outputs()

Use pytorch device_name: mps
Load pretrained SentenceTransformer: all-MiniLM-L6-v2
HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json "HTTP/1.1 200 OK"
HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json "HTTP/1.1 307 Temporary Redirect"
HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/config_sentence_transformers.json "HTTP/1.1 200 OK"
HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json "HTTP/1.1 307 Temporary Redirect"
HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"
HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/config.json "HTTP/1.1 200 OK"
HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json "HTTP/1.1 307 Temporary Redirect"
HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/tokenizer_config.json "HTTP/1.1 200 OK"
HTTP Request: GET https://hug

Batches:   0%|          | 0/500 [00:00<?, ?it/s]

Embedding shape: (32000, 384)
Use pytorch device_name: mps
Load pretrained SentenceTransformer: all-MiniLM-L6-v2
HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json "HTTP/1.1 200 OK"
HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json "HTTP/1.1 307 Temporary Redirect"
HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/config_sentence_transformers.json "HTTP/1.1 200 OK"
HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json "HTTP/1.1 307 Temporary Redirect"
HTTP Request: HEAD https://huggingface.co/api

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"
HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/config.json "HTTP/1.1 200 OK"
HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json "HTTP/1.1 307 Temporary Redirect"
HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/tokenizer_config.json "HTTP/1.1 200 OK"
HTTP Request: GET https://hug

Batches:   0%|          | 0/500 [00:00<?, ?it/s]

Embedding shape: (32000, 384)


In [6]:
rec_tfidf = ContentRecommender(
    df, 
    tfidf_matrix=tfidf_matrix,
    feature_matrices=fe.feature_matrices,
)

rec_efe_tag = ContentRecommender(
    df, 
    tfidf_matrix=tfidf_matrix,
    feature_matrices=fe.feature_matrices,
    embeddings=tag_embeddings
)

rec_efe_des = ContentRecommender(
    df, 
    tfidf_matrix=tfidf_matrix,
    feature_matrices=fe.feature_matrices,
    embeddings=des_embeddings
)

rec_tfidf.compute_weighted_rating()
rec_efe_tag.compute_weighted_rating()
rec_efe_des.compute_weighted_rating()

In [7]:
sample_titles = df['title'].sample(1).tolist()

for title in sample_titles:
    print(f"Target Title: {title}")
    print("="*110)

    print(">> [A] Combined (TF-IDF Soup)")
    res_a = rec_tfidf.recommend(title, top_k=5, approach='combined')
    display(res_a)

    print(">> [B] Separate (TF-IDF Weighted Features)")
    res_b = rec_tfidf.recommend(title, top_k=5, approach='separate')
    display(res_b)

    print(">> [C] Tag Embedding (SBERT Soup)")
    res_c = rec_efe_tag.recommend(title, top_k=5, approach='embedding')
    display(res_c)

    print(">> [D] Description Embedding (SBERT Plot)")
    res_d = rec_efe_des.recommend(title, top_k=5, approach='embedding')
    display(res_d)

Target Title: My King
>> [A] Combined (TF-IDF Soup)


,title,type,genres,vote_average,weighted_rating,similarity,final_score
13105,TOGEFILM - Mei Mei,Movie,"Drama, Romance",10.0,5.71,0.458927,0.438730
15658,TACHÉE,Movie,"Drama, Romance",0.0,5.69,0.458927,0.437836
12704,PSK Belia,Movie,"Drama, Romance",0.0,5.69,0.458927,0.437836
13340,Salome,Movie,"Drama, Romance",10.0,5.70,0.415120,0.403237
13932,STW,Movie,"Drama, Romance",0.0,5.69,0.415120,0.402790


>> [B] Separate (TF-IDF Weighted Features)


,title,type,genres,vote_average,weighted_rating,similarity,final_score
6329,From the Land of the Moon,Movie,"Drama, Romance",6.6,6.07,0.445888,0.444406
944,Deep in the Woods,Movie,"Drama, Romance",5.7,5.69,0.460302,0.438935
5163,One Wild Moment,Movie,"Drama, Romance",5.8,5.77,0.439969,0.426248
4763,"Rio, I Love You",Movie,"Romance, Drama",4.9,5.61,0.439428,0.418656
7712,Gauguin: Voyage to Tahiti,Movie,"Drama, Romance",5.7,5.69,0.433904,0.417816


>> [C] Tag Embedding (SBERT Soup)


,title,type,genres,vote_average,weighted_rating,similarity,final_score
2398,Rust and Bone,Movie,"Drama, Romance",7.200,6.85,0.748224,0.721174
3682,The Firefly,Movie,"Drama, Fantasy, Romance",6.899,5.95,0.739213,0.673697
14972,Mahal ko ang Mahal Mo,Movie,Drama,6.200,5.69,0.749543,0.670328
20254,Vitória,TV Show,Drama,0.000,5.69,0.741419,0.663829
20130,Guapas,TV Show,Drama,4.000,5.68,0.741745,0.663642


>> [D] Description Embedding (SBERT Plot)


,title,type,genres,vote_average,weighted_rating,similarity,final_score
25402,After Life,TV Show,"Comedy, Drama",7.900,7.19,0.570693,0.594362
628,Tony,Movie,"Horror, Drama, Thriller",5.900,5.72,0.588592,0.542909
24955,The Good Cop,TV Show,"Comedy, Drama",6.200,5.74,0.494146,0.468247
8647,Burn Out,Movie,"Action, Thriller, Crime",6.700,6.20,0.454701,0.457273
7328,Unforgettable,Movie,"Drama, Thriller",5.891,5.81,0.460866,0.444756


In [8]:
evaluator = Evaluator(
    df=rec_efe_tag.df,
    tfidf_matrix=tfidf_matrix,
    feature_matrices=fe.feature_matrices,
    feature_weights={'genres': 0.4, 'cast': 0.3, 'description': 0.2, 'director': 0.1},
    embeddings=tag_embeddings
)

In [9]:
results_h1 = evaluator.test_h1_discriminative_power(sample_size=500, top_k=10)

Hypothesis 1: Discriminative Power
Mean genre-only variance:    0.000132
Mean multi-feature variance: 0.000899
Wilcoxon statistic: 12458.00
P-value: 9.42e-51
Significant at α=0.05: True


In [10]:
results_h2 = evaluator.test_h2_feature_independence(sample_size=1000)

Hypothesis 2: Feature Independence
Sample pairs: 1000
Pair                           Spearman ρ      P-value Interpretation
---------------------------------------------------------------------------
genres vs cast                     0.0247     4.34e-01 Independent
genres vs description              0.0926     3.38e-03 Independent
genres vs director                    nan          nan Highly correlated (redundant)
cast vs description               -0.0243     4.44e-01 Independent
cast vs director                      nan          nan Highly correlated (redundant)
description vs director               nan          nan Highly correlated (redundant)


In [11]:
results_h3 = evaluator.test_h3_ranking_quality(sample_size=500, top_k=20)

Hypothesis 3: Ranking Quality
Kendall's Tau (raw vs weighted ranking):
  Mean tau: 0.0640
  (1.0 = identical, 0.0 = no correlation)
Avg vote_count in top-5 recommendations:
  Raw vote_average ranking:  849.6
  Weighted rating ranking:   1237.3
Wilcoxon test (vote confidence):
  Statistic: 57.50
  P-value: 1.04e-79
  Significant at α=0.05: True


In [12]:
results_h4 = evaluator.test_h4_method_distinctness(sample_size=500, top_k=10)

Hypothesis 4: Method Distinctness (Novelty)
Mean Jaccard Similarity (TF-IDF vs SBERT): 0.0339
Interpretation: High Distinctness. SBERT is finding completely new items.


In [13]:
results_h5 = evaluator.test_h5_score_distribution(sample_size=500)

Hypothesis 5: Score Distribution (The 'Loudness' Problem)
Mean Top-50 TF-IDF Score:   0.4184
Mean Top-50 Embedding Score: 0.6317
KS Statistic: 0.7012
P-value: 0.00e+00
Significant Difference: True
Result: Embeddings are significantly 'louder'. Normalization is recommended for Hybrid.


In [14]:
results_h6 = evaluator.test_h6_hybrid_quality_lift(sample_size=500, top_k=10)

Hypothesis 6: Hybrid Quality Lift
Avg Rating (TF-IDF Top-10): 5.82
Avg Rating (Hybrid Top-10): 5.83
Wilcoxon Statistic: 53362.50
P-value: 4.34e-02
Result: Hybrid strategy significantly improves recommendation quality.


# Best Approach Comparison

Compare all recommendation approaches on a sample of titles to determine which produces the highest quality recommendations (by mean `final_score` and `weighted_rating`).

In [19]:
np.random.seed(42)
sample_titles = df['title'].sample(200).tolist()

approaches = {
    'Combined (TF-IDF Soup)':       (rec_tfidf, 'combined'),
    'Separate (TF-IDF Weighted)':   (rec_tfidf, 'separate'),
    'Embedding (SBERT Tag)':        (rec_efe_tag, 'embedding'),
    'Embedding (SBERT Desc)':       (rec_efe_des, 'embedding'),
    'Hybrid (TF-IDF + SBERT Tag)':  (rec_efe_tag, 'hybrid'),
    'Hybrid (TF-IDF + SBERT Desc)': (rec_efe_des, 'hybrid'),
}

comparison = []
for name, (recommender, approach) in approaches.items():
    scores = []
    ratings = []
    for title in sample_titles:
        result = recommender.recommend(title, top_k=10, approach=approach)
        if result is not None and 'final_score' in result.columns:
            scores.append(result['final_score'].mean())
            ratings.append(result['weighted_rating'].mean())
    comparison.append({
        'Approach': name,
        'Mean Final Score': np.mean(scores),
        'Mean Weighted Rating': np.mean(ratings),
        'Samples': len(scores),
    })

comparison_df = pd.DataFrame(comparison).sort_values('Mean Final Score', ascending=False)
display(comparison_df)

best_approach = comparison_df.iloc[0]['Approach']
print(f"\nBest approach: {best_approach}")

Multiple matches for 'Father of the Bride'. Using first match. Specify content_type or year to disambiguate.
                     title     type                  genres  release_year
12555  Father of the Bride    Movie  Romance, Comedy, Drama          2022
23100  Father of the Bride  TV Show   Family, Drama, Comedy          2017
Multiple matches for 'Downton Abbey'. Using first match. Specify content_type or year to disambiguate.
               title     type                   genres  release_year
9338   Downton Abbey    Movie  Drama, Romance, History          2019
16046  Downton Abbey  TV Show                    Drama          2010
Multiple matches for 'Bad Education'. Using first match. Specify content_type or year to disambiguate.
               title     type                 genres  release_year
9530   Bad Education    Movie  Drama, Crime, History          2019
18332  Bad Education  TV Show                 Comedy          2012
Multiple matches for 'Brother and Sister'. Using first 

,Approach,Mean Final Score,Mean Weighted Rating,Samples
2,Embedding (SBERT Tag),0.615164,5.892275,200
3,Embedding (SBERT Desc),0.531876,5.896410,200
4,Hybrid (TF-IDF + SBERT Tag),0.513694,5.827025,200
0,Combined (TF-IDF Soup),0.454773,5.820295,200
5,Hybrid (TF-IDF + SBERT Desc),0.448870,5.870290,200
1,Separate (TF-IDF Weighted),0.410687,5.923760,200



Best approach: Embedding (SBERT Tag)


In [21]:
test_cases = [
    ('Inception', None, None),
    ('Frozen', 'Movie', 2013),
    ('Parasite', None, None),
    ('Stranger Things', 'TV Show', None),
    ('Get Out', 'Movie', 2017)
]

for title, ctype, year in test_cases:
    print(f"\n{'='*60}")
    print(f"Input: {title} ({ctype or 'Any'}, {year or 'Any'})")
    print('='*60)
    
    matches = rec_efe_tag.df[rec_efe_tag.df['title'].str.lower() == title.lower()]
    if ctype:
        matches = matches[matches['type'] == ctype]
    if year:
        matches = matches[matches['release_year'] == year]
    
    if matches.empty:
        print("Not found")
        continue
    
    idx = matches.index[0]
    
    print(f"Genres: {rec_efe_tag.df.loc[idx, 'genres']}")
    print(f"Cast: {str(rec_efe_tag.df.loc[idx, 'cast'])[:80]}")
    print(f"Description: {str(rec_efe_tag.df.loc[idx, 'description'])[:120]}...")
    print()
    
    indices, scores = rec_efe_tag._get_similar_separate(idx, top_k=5)
    results = rec_efe_tag.df.loc[indices, ['title', 'type', 'genres', 'vote_average', 'weighted_rating']].copy()
    results['similarity'] = scores
    display(results)


Input: Inception (Any, Any)
Genres: Action, Science Fiction, Adventure
Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ken Watanabe, Tom Hardy, Elliot Page
Description: Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chanc...



,title,type,genres,vote_average,weighted_rating,similarity
13066,The Creator,Movie,"Science Fiction, Action, Adventure",7.067,6.91,0.455191
14006,Venom: The Last Dance,Movie,"Action, Science Fiction, Adventure",6.786,6.65,0.452731
11010,Venom: Let There Be Carnage,Movie,"Science Fiction, Action, Adventure",6.779,6.74,0.452364
5004,Mad Max: Fury Road,Movie,"Action, Adventure, Science Fiction",7.600,7.57,0.446022
4000,Interstellar,Movie,"Adventure, Drama, Science Fiction",8.500,8.47,0.440163



Input: Frozen (Movie, 2013)
Genres: Animation, Family, Adventure, Fantasy
Cast: Kristen Bell, Idina Menzel, Jonathan Groff, Josh Gad, Livvy Stubenrauch
Description: Young princess Anna of Arendelle dreams about finding true love at her sister Elsa’s coronation. Fate takes her on a dan...



,title,type,genres,vote_average,weighted_rating,similarity
9013,Frozen II,Movie,"Family, Animation, Adventure, Comedy, Fantasy",7.2,7.14,0.748688
5147,Frozen Fever,Movie,"Animation, Family, Adventure, Comedy",6.8,6.59,0.676759
7109,Olaf's Frozen Adventure,Movie,"Animation, Family, Adventure, Comedy, Fantasy",6.5,6.32,0.632628
10155,Once Upon a Snowman,Movie,"Animation, Family, Comedy, Fantasy",6.9,6.37,0.536538
7188,LEGO Frozen Northern Lights,Movie,"Animation, Family",6.6,5.86,0.536267



Input: Parasite (Any, Any)
Genres: Comedy, Thriller, Drama
Cast: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Choi Woo-shik, Park So-dam
Description: All unemployed, Ki-taek's family takes peculiar interest in the wealthy and glamorous Parks for their livelihood until t...



,title,type,genres,vote_average,weighted_rating,similarity
13639,Killing Romance,Movie,"Thriller, Comedy",6.4,5.71,0.423111
4154,Wild Tales,Movie,"Drama, Thriller, Comedy",7.9,7.66,0.412600
12809,Emergency,Movie,"Comedy, Drama, Thriller",6.1,5.83,0.412277
13161,Saltburn,Movie,"Drama, Comedy, Thriller",7.0,6.81,0.403432
14316,Miller's Girl,Movie,"Thriller, Comedy, Drama",6.4,6.15,0.400000



Input: Stranger Things (TV Show, Any)
Genres: Drama, Science Fiction, Fantasy, Mystery
Cast: Millie Bobby Brown, Finn Wolfhard, Gaten Matarazzo, Caleb McLaughlin, Sadie Sink
Description: When a young boy vanishes, a small town uncovers a mystery involving secret experiments, terrifying supernatural forces,...



,title,type,genres,vote_average,weighted_rating,similarity
30151,Don't Come Home,TV Show,"Mystery, Drama, Science Fiction, Fantasy",7.700,5.95,0.428717
19076,Under the Dome,TV Show,"Drama, Mystery, Science Fiction, Fantasy",7.162,7.01,0.425927
25859,Swamp Thing,TV Show,"Mystery, Science Fiction, Fantasy, Drama",7.443,6.93,0.423050
28918,Archive 81,TV Show,"Drama, Mystery, Science Fiction, Fantasy",7.200,6.56,0.423032
25201,Creepshow,TV Show,"Mystery, Science Fiction, Fantasy",7.500,6.43,0.422752



Input: Get Out (Movie, 2017)
Genres: Mystery, Thriller, Horror
Cast: Daniel Kaluuya, Allison Williams, Catherine Keener, Bradley Whitford, Caleb Land
Description: Chris and his girlfriend Rose go upstate to visit her parents for the weekend. At first, Chris reads the family's overly...



,title,type,genres,vote_average,weighted_rating,similarity
72,The Last Exorcism,Movie,"Horror, Thriller, Mystery",5.700,5.70,0.453654
13979,Older Gods,Movie,"Horror, Mystery, Thriller",6.400,5.77,0.422647
9086,Us,Movie,"Horror, Thriller",6.964,6.90,0.421646
12126,Nope,Movie,"Horror, Mystery, Science Fiction",6.800,6.70,0.415572
13445,There's Something Wrong with the Children,Movie,"Horror, Mystery, Thriller",5.700,5.69,0.413915


In [29]:
import pickle
import json
import scipy.sparse as sp

class ModelExporter:
    def __init__(self, df, tfidf_matrix, tfidf, feature_matrices, feature_weights,
                 embeddings=None, hybrid_weights=None, best_approach=None):
        self.df = df
        self.tfidf_matrix = tfidf_matrix
        self.tfidf = tfidf
        self.feature_matrices = feature_matrices
        self.feature_weights = feature_weights
        self.embeddings = embeddings
        self.hybrid_weights = hybrid_weights
        self.best_approach = best_approach

    def export(self, output_dir='model_artifacts'):
        os.makedirs(output_dir, exist_ok=True)

        self.df.to_parquet(f'{output_dir}/processed_data.parquet', index=False)
        sp.save_npz(f'{output_dir}/tfidf_matrix.npz', self.tfidf_matrix)

        with open(f'{output_dir}/tfidf_vectorizer.pkl', 'wb') as f:
            pickle.dump(self.tfidf, f)

        for name, matrix in self.feature_matrices.items():
            sp.save_npz(f'{output_dir}/feature_{name}.npz', matrix)

        with open(f'{output_dir}/feature_weights.pkl', 'wb') as f:
            pickle.dump(self.feature_weights, f)

        if self.embeddings is not None:
            np.save(f'{output_dir}/embeddings.npy', self.embeddings)
            
        config = {'best_approach': self.best_approach}
        if self.hybrid_weights is not None:
            config['hybrid_weights'] = list(self.hybrid_weights)
        with open(f'{output_dir}/config.json', 'w') as f:
            json.dump(config, f, indent=2)

        print(f"Exported to {output_dir}/:")
        for fname in sorted(os.listdir(output_dir)):
            size = os.path.getsize(f'{output_dir}/{fname}') / 1024
            print(f"  {fname}: {size:.1f} KB")

In [30]:
exporter = ModelExporter(
    df=rec_efe_tag.df,
    tfidf_matrix=tfidf_matrix,
    tfidf=tfidf,
    feature_matrices=fe.feature_matrices,
    feature_weights={'genres': 0.4, 'cast': 0.3, 'description': 0.2, 'director': 0.1},
    embeddings=tag_embeddings,
    hybrid_weights=(0.5, 0.5),
    best_approach=best_approach
)
exporter.export()

Exported to model_artifacts/:
  config.json: 0.1 KB
  embeddings.npy: 48000.1 KB
  feature_cast.npz: 1206.7 KB
  feature_description.npz: 6101.1 KB
  feature_director.npz: 130.6 KB
  feature_genres.npz: 206.7 KB
  feature_weights.pkl: 0.1 KB
  processed_data.parquet: 17210.9 KB
  tfidf_matrix.npz: 7445.8 KB
  tfidf_vectorizer.pkl: 378.2 KB
